# [L03: Grid Approximation TSP](https://docs.google.com/document/d/1L8J0AKVoiDuaLXy4FjzH9eGUq8B4W-kQWj6eoGL0G40/edit?usp=sharing)
Written by Dr. Jan Pearce, Berea College

Complete the code by meeting all linked requirements and fixing all FIXMEs

## Your Name: Anh Ngo

**SUMMARY**: A brief summary description of the design and implementation, including how much your initial design plan evolved, the final result you achieved and the amount of time you spent as a programmer or programmers in accomplishing these results. This should be no more than two paragraphs.

I did plan to complete the lab but I was not able to do so even with extensive studying. However, 70% of my plan evolved and the final result was the methods that I was asked to complete. I spent 10 hours in this lab.

**PERSONNEL**: A description of who you worked with and on what sections.  It is expected that all collaboration is done as pair programming together. Also, note these collaborations in the code section itself.

I worked by myself in this lab.

**CHALLENGES**: Descriptions of the largest challenges you overcame and what made them challenging.

The largest challenge is to understand the intuition behind the requirement. I was trying my best to research on it but there was limitted resources I could find. I was not able to make it to the office hours so this lab was very challenging to me. 

**INNOVATIONS**: Any innovations that were not specifically required by the assignment. These are not required, but should be highlighted if included.

I was not making any innovations in this lab. 

**TESTING**: Describe how you tested this work.

I printed things out line by line to see whether it worked like what I expected.

**ERRORS**: A list in bulleted form of all known errors and deficiencies.

My kernel keeps sending errors that it has died due to systematic errors. I also was not able to separate nodes on the partition line to different sections. However, all required methods should work properly.

**COMMENTS**: A paragraph or so of your own comments on and reactions to the Lab. 

I think this lab is very hard and challenging by the fact that I am not very experienced with recursion myself. That should be something I need to improve in the future. However, it is a great learning experience. 

## Import Libraries

In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import math
from urllib.request import urlopen
import random
import itertools

### Read data

In [3]:
def get_new_node_coords(numnodes, scale=1):
    nodeStr = f'{numnodes}\n'
    coords = []
    for i in range(numnodes):
        x, y = random.randint(1, numnodes//scale), random.randint(1, numnodes//scale)
        while (x, y) in coords:
            x, y = random.randint(1, numnodes//scale), random.randint(1, numnodes//scale)
        coords.append((x, y))
        nodeStr += f'{x} {y}\n'
    return nodeStr
    

# tspdata = urlopen("http://cs.berea.edu/courses/csc445/data/tsp-48.txt").read().decode('ASCII')
# tspdata = urlopen("http://cs.berea.edu/courses/csc445/data/tsp-small.txt").read().decode('ASCII')
tspdata = get_new_node_coords(20)
print(tspdata)

20
10 10
13 11
17 8
8 20
13 1
4 5
8 14
3 1
12 1
18 10
7 12
15 8
19 10
6 12
10 2
12 9
13 4
14 10
8 3
8 6



## TSP Class

In [1]:
"""

Given certain input, it uses the grid approximation algorithm to plot a graph and output a file of the solution path
"""

class TSP:

    def __init__(self):
        """ TSP class containing data needed for networkx graph creation, file reading and file writing. 
            Creates a graph for the traveling salesperon problem
        """

        self.dimension = None
        self.nodeData = []
        self.graph = nx.DiGraph()
        self.coords = {}
        self.route = []

    def extractData(self, inputFile=""):
        """ Iterates through the input file and append each coordinate into the node list.
        Keyword Arguments:
            inputFile {str} -- a string for the file name (default: {""})
        """
        WTSPC = ['\n', '\r', '\t', ' ']
        inputData = inputFile.split()
        cleanData = [int(i) for i in inputData if i not in WTSPC]

        self.dimension = cleanData[0]

        for i in range(1, len(cleanData), 2):
            nodeX = cleanData[i]
            nodeY = cleanData[i + 1]
            self.nodeData.append([nodeX, nodeY])

    def setUpCoords(self):
        """ Puts the coordinates for each node into a dictionary as a tuple with the node as the key """
        for i in range(len(self.nodeData)):
            self.coords[i] = self.nodeData[i]

    def getDirection(self, a, b, c):
        """ Determines if the angle made by the path a-b-c is clockwise or
            couter clockwise
        Arguments:
            a {int} -- the integer index of a node
            b {int} -- the integer index of a node
            c {int} -- the integer index of a node
        Returns:
            {int} -- 1 if a->b->c is clockwise, -1 if counterclockwise
        """
        ax, ay = self.coords[a][0], self.coords[a][1]
        bx, by = self.coords[b][0], self.coords[b][1]
        cx, cy = self.coords[c][0], self.coords[c][1]
        val = (by-ay)*(cx-bx)-(bx-ax)*(cy-by)
        if val < 0:
            return -1
        elif val > 0:
            return 1
        return 0

    def calculateDistance(self, a, b): 
        """ Calculates & returns the distance between points a and b
        Arguments:
            a {tuple (int)} -- tuple of numbers for the x and y coordinate of a node
            b {tuple (int)} -- tuple of numbers for the x and y coordinate of a node
        Returns:
            {float} -- the distance between the two points using the distance formula
        """
        x1 = float(a[0])
        y1 = float(a[1])
        x2 = float(b[0])
        y2 = float(b[1])
        xSquared = math.pow(x2 - x1, 2)
        ySquared = math.pow(y2 - y1, 2)
        c = round(math.sqrt(xSquared + ySquared), 2)
        return c

    def calculateTotalDistance(self, route=None):
        """
        """
        if route == None:
            route = self.route

        total = 0
        for i in range(len(route) - 1):
            n1 = self.coords[route[i]]
            n2 = self.coords[route[i+1]]
            total += self.calculateDistance(n1, n2)
        return total

    def getAllTours(self, nodes):
        """ Calculates all possible tours using nodes from 'nodes'
        Arguments:
            nodes {list (int)} -- List of node numbers
        Returns:
            A list of tuples, each tuple representing a single tour
        """
        tours = list(itertools.permutations(nodes))
        return [list(t) + [t[0]] for t in tours]

    def isClockwise(self, route):
        """ Decides if a route contains only clockwise angles
            NOTE: If route is of size 5, there can be a single angle that is
            the opposite of the others.
        Arguments:
            route {list (int)} -- List of node numbers on route
        Returns:
            {bool} True if all angles are clockwise, False otherwise
        """
        edges = [(route[i], route[i+1], route[i+2]) for i in range(len(route) - 2)]
        return all([self.getDirection(a, b, c) in [0, 1] for (a, b, c) in edges])

    def isCounterclockwise(self, route):
        """ Decides if a route contains only counterclockwise angles
            NOTE: If route is of size 5, there can be a single angle that is
            the opposite of the others.
        Arguments:
            route {list (int)} -- List of node numbers on route
        Returns:
            {bool} True if all angles are clockwise, False otherwise
        """
        edges = [(route[i], route[i+1], route[i+2]) for i in range(len(route) - 2)]
        return all([self.getDirection(a, b, c) in [0, -1] for (a, b, c) in edges])

    def horizontalPartition(self, nodes, xval):
        """ Splits the list 'nodes' into two lists, left and right, such that
            all nodes in 'left' have an x-coordinate less than or equal to
            xval, and all nodes in 'right' have an x-coordinate greater than or
            equal to xval.
            
            *** NOTE: THIS DOES NOT BREAK TIES.  ANY NODE WITH AN X-COORDINATE
            EQUAL TO XVAL WILL GO TO BOTH LISTS.  YOU MUST DECIDE HOW TO
            BREAK THE TIE.
        Arguments:
            nodes {list (int)} -- the list of nodes to partition
            xval {int} -- the x-coordinate boundary between partitions
        Returns:
            Two lists of ints, 'left' and 'right'.  If any nodes lie on the
            boundary, they will be included in both lists.
        """
        bounds = self.getBounds(list(range(self.graph.number_of_nodes())))
        xmin, xmax = bounds['xmin'], bounds['xmax']
        ymin, ymax = bounds['ymin'], bounds['ymax']

        left = self.getNodesBetween(nodes, xmin, xval, ymin, ymax)
        right = self.getNodesBetween(nodes, xval, xmax, ymin, ymax)
        return left, right

    def verticalPartition(self, nodes, yval):
        """ Splits the list 'nodes' into two lists, top and bottom, such that
            all nodes in 'top' have a y-coordinate greater than or equal to
            yval, and all nodes in 'bottom' have a y-coordinate less than or
            equal to yval.
            
            *** NOTE: THIS DOES NOT BREAK TIES.  ANY NODE WITH A Y-COORDINATE
            EQUAL TO YVAL WILL GO TO BOTH LISTS.  YOU MUST DECIDE HOW TO
            BREAK THE TIE.
        Arguments:
            nodes {list (int)} -- the list of nodes to partition
            yval {int} -- the y-coordinate boundary between partitions
        Returns:
            Two lists of ints, 'top' and 'bottom'.  If any nodes lie on the
            boundary, they will be included in both lists.
        """
        bounds = self.getBounds(list(range(self.graph.number_of_nodes())))
        xmin, xmax = bounds['xmin'], bounds['xmax']
        ymin, ymax = bounds['ymin'], bounds['ymax']

        top = self.getNodesBetween(nodes, xmin, xmax, yval, ymax)
        bottom = self.getNodesBetween(nodes, xmin, xmax, ymin, yval)
        return top, bottom

    def getBounds(self, nodes):
        """ Gets the bounding box containing every node in 'nodes'
        Arguments:
            nodes {list (int)} -- List of node numbers to bound
        Returns:
            dict {(str) : (int)} mapping the strings 'xmax', 'xmin', 'ymax', 'ymin'
            to the corresponding bounding values
        """
        allcoords = [self.coords[i] for i in nodes]
        myx = [i[0] for i in allcoords]
        myy = [i[1] for i in allcoords]
        return {'xmax': max(myx), 'xmin': min(myx),
                'ymax': max(myy), 'ymin': min(myy)}

        # To use this elsewhere in the code, do something like:
        # bounds = self.getBounds([0, 1, 2, 3, 4])
        # my_minimum_x_value = bounds['xmin']
        # my_maximum_y_value = bounds['ymax']

    def getNodesBetween(self, nodes, xmin, xmax, ymin, ymax):
        """ Returns all nodes within the specified bounding box
        Argments:
            nodes {list (int)} -- the list of nodes to search within
            xmin {int} -- The minimum x-value for any node
            xmax {int} -- The maximum x-value for any node
            ymin {int} -- The minimum y-value for any node
            ymax {int} -- The maximum y-value for any node
        Returns:
            {list (int)} -- A list of node numbers that fall within the
            specified bounding box
        """
        ret = []
        for i in nodes:
            myx = self.coords[i][0]
            myy = self.coords[i][1]
            if (xmin <= myx <= xmax) and (ymin <= myy <= ymax):
                ret.append(i)
        return ret

    def getGreedyRoute(self, start): 
        """Builds a route using the Greedy algorithm
           Stores route in self.route
        Arguments:
            start {int} -- the node to start at, not a list index
        """
        self.route = [start]
        nVisited = [i for i in self.graph.nodes() if i != start]
        current = start
        while nVisited:
            distances = {i: self.calculateDistance(self.coords[current], self.coords[i]) for i in nVisited}
            minDist = 1000000000000
            closestNode = -1
            for node in distances:
                if distances[node] < minDist:
                    minDist = distances[node]
                    closestNode = node
            self.route.append(closestNode)
            nVisited.remove(closestNode)
            current = closestNode
            #print(self.route)
        self.route.append(start)
        
    def gridApprox(self, currentNodes=None): 
        """ Uses the grid approximation algorithm. Gets the route order and 
            adds the starting place to the end of the route.
        Keyword Arguments:
            currentNodes {list (int)} -- the list of node indexes within the
            current segment of the grid.  If no value is provided, currentNodes
            is assigned the set of all nodes in the graph.
        """
        if currentNodes is None:
            currentNodes = list(range(self.graph.number_of_nodes()))
        
        if len(currentNodes) in [3, 4, 5]:
            print('Base case')
            tours = self.getAllTours(currentNodes)
            #print(tours)
            myTour = None
            for t in tours:
                if (self.isClockwise(t)):
                    myTour = t
                    break
            print(myTour)
            return myTour
        
        else:
            
            bounds = self.getBounds(currentNodes)
            midYval = (bounds['ymax'] + bounds['ymin']) / 2
            n1, n2 = self.verticalPartition(currentNodes, midYval)
            #print(n1, n2)
            t1 = self.gridApprox(currentNodes = n1)
            t2 = self.gridApprox(currentNodes = n2)
            self.combineTours(t1, t2, 'vertical')
            
            midXval = (bounds['xmax'] + bounds['xmin']) / 2
            m1, m2 = self.horizontalPartition(currentNodes, midXval)
            u1 = self.gridApprox(currentNodes = m1)
            u2 = self.gridApprox(currentNodes = m2)
            self.combineTours(u1, u2, 'horizontal')
            
    def combineTours(self, tour1, tour2, split='horizontal'):
        """ Combines two tours by finding a place to rewire two edges.
        Arguments:
            tour1 {list (int)} --
            tour2 {list (int)} --
            split {str} -- Either 'horizontal' or 'vertical'.  Helps you keep
            track of how you split the two chunks you are now combining.
        Returns:
            A list of ints representing the new combined tour.
        """
        shortestDist = float('inf')
        shortestPath = None
        if tour1:
            for i in tour1:
                if tour2:
                    for j in tour2:
                        currentDist = self.calculateDistance(self.coords[i], self.coords[j])
                        if currentDist < shortestDist:
                            shortestDist = currentDist
                            shortestPath = (i, j)
                else:
                    break
        
        if shortestPath:
            return tour1[:tour1.index(i) + 1] + tour2[tour2.index(j):] + tour2[:tour2.index(j)] + tour1[tour1.index(i)+1:]
        else:
            return
        
    def addNodes(self): 
        """ Adds nodes to the networkx graph
        """
        for node in self.coords:
            self.graph.add_node(node, pos=self.coords[node])

    def addEdges(self, route=None): 
        """Adds edges to a networkx graph
        """
        if route == None:
            route = self.route
        x = 0
        while x < len(route) - 1:
            self.graph.add_edge(route[x], route[x+1],
                                weight=self.calculateDistance(self.coords[route[x]], self.coords[route[x+1]]))
            x += 1

    def showGraph(self):
        """Uses the imported libraries to display the graph
        """
        # get labels and coordinates for the graph
        coords = nx.get_node_attributes(self.graph, 'pos')
        labels = nx.get_edge_attributes(self.graph, 'weight')
        # draw nodes in their coordinates
        nx.draw(self.graph, coords)
        nx.draw_networkx_labels(self.graph, coords)
        # draw edges with their weights
        nx.draw_networkx_edge_labels(self.graph, coords, edge_labels=labels)
        plt.show()

    def findSplit(self):
        nodes = list(self.graph.nodes())
        bounds = self.getBounds(nodes)
        print(bounds)
        ymid = (bounds['ymin'] + bounds['ymax']) / 2
        part1, part2 = self.verticalPartition(nodes, ymid)
        print(part1)
        print(part2)
        ymid -= 2
        part1, part2 = self.verticalPartition(nodes, ymid)
        print(part1)
        print(part2)
        b = self.getBounds(part1)
        print(b)



def main():
    # Create TSP object
    tsp = TSP()

    # Perform initial setup with data generated above
    tsp.extractData(tspdata)
    tsp.setUpCoords()
    tsp.addNodes()

    tsp.findSplit()

    #Call the grid approximation method
    tsp.gridApprox()
    tsp.addEdges()
    tsp.showGraph()


main()

NameError: name 'nx' is not defined

## Integrity statement
Please briefly describe all references used, all help you received and all help you gave to others in completing this assignment. Be sure to say that you got no help if you got none.